<a href="https://colab.research.google.com/github/devikaajay/Startup_Success_Prediction/blob/main/startup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.impute import SimpleImputer
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [ ]:
import pandas as pd
import numpy as np

# 1. Load the CSV file
try:
    df = pd.read_csv('/content/big_startup_secsees_dataset.csv', encoding='utf-8')
except FileNotFoundError:
    print("Error: 'big_startup_secsees_dataset.csv' not found. Please upload your file and ensure the name is correct.")
    df = pd.DataFrame()

REQUIRED_COLUMNS = [
    'name',
    'homepage_url',
    'category_list',
    'funding_total_usd',
    'status',
    'country_code',
    'state_code',
    'region',
    'city',
    'funding_rounds'
]

# --- Data Cleaning and Selection ---

if df.empty:
    print("DataFrame is empty. Skipping further processing.")
else:

    missing_cols = [col for col in REQUIRED_COLUMNS if col not in df.columns]
    if missing_cols:
        print(f"Error: The following required columns are missing from the uploaded file: {', '.join(missing_cols)}")

        df = df[[col for col in REQUIRED_COLUMNS if col in df.columns]]
    else:
        df = df[REQUIRED_COLUMNS]


    print(f"Original shape after column selection: {df.shape}")

    df_no_all_duplicates = df.drop_duplicates()
    print(f"Shape after removing all-column duplicates: {df_no_all_duplicates.shape}")

    df_unique_names = df_no_all_duplicates.drop_duplicates(subset=['name'], keep='first')
    print(f"Shape after enforcing unique company names: {df_unique_names.shape}")

    df_cleaned = df_unique_names.dropna()
    print(f"Shape after removing all rows with null values: {df_cleaned.shape}")


    print("\nFirst 5 rows of the cleaned data:")
    print(df_cleaned.head())

    CLEANED_FILE_NAME = 'cleaned_startup_data_for_prediction.csv'
    df_cleaned.to_csv(CLEANED_FILE_NAME, index=False)
    print(f"\nCleaned data saved to: {CLEANED_FILE_NAME}")

Original shape after column selection: (66368, 10)
Shape after removing all-column duplicates: (66367, 10)
Shape after enforcing unique company names: (66103, 10)
Shape after removing all rows with null values: (52551, 10)

First 5 rows of the cleaned data:
                     name                       homepage_url  \
0                   #fame                 http://livfame.com   
1                :Qounter             http://www.qounter.com   
3                 0-6.com                 http://www.0-6.com   
4        004 Technologies  http://004gmbh.de/en/004-interact   
6  Ondine Biomedical Inc.               http://ondinebio.com   

                                       category_list funding_total_usd  \
0                                              Media          10000000   
1  Application Platforms|Real Time|Social Network...            700000   
3                                        Curated Web           2000000   
4                                           Software         

In [ ]:
df_cleaned.shape

(52551, 10)

In [ ]:
import pandas as pd
import re
import numpy as np

print(f"Starting shape for advanced cleaning: {df_cleaned.shape}")


initial_shape = df_cleaned.shape[0]
name_mask = df_cleaned['name'].astype(str).str.match(r'^[a-zA-Z]')
df_cleaned = df_cleaned[name_mask]
print(f"Shape after removing names starting with non-letters: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")

initial_shape = df_cleaned.shape[0]
numeric_state_mask = df_cleaned['state_code'].astype(str).str.isnumeric()

df_cleaned = df_cleaned[~numeric_state_mask]
print(f"Shape after removing numeric state codes: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")

initial_shape = df_cleaned.shape[0]
single_letter_mask = df_cleaned['name'].astype(str).str.len() == 1

df_cleaned = df_cleaned[~single_letter_mask]
print(f"Shape after removing single-letter names: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")


initial_shape = df_cleaned.shape[0]
df_cleaned['funding_total_usd'] = pd.to_numeric(
    df_cleaned['funding_total_usd'],
    errors='coerce'
)

df_cleaned.dropna(subset=['funding_total_usd'], inplace=True)
print(f"Shape after validating/cleaning funding values: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")

TARGET_ROWS = 3000
initial_shape_trunc = df_cleaned.shape[0]

if initial_shape_trunc > TARGET_ROWS:
    df_cleaned = df_cleaned.head(TARGET_ROWS)
    print(f"Dataset truncated to the top {TARGET_ROWS} rows.")
elif initial_shape_trunc < TARGET_ROWS:
    print(f"Warning: Only {initial_shape_trunc} rows remain after cleaning. No truncation performed.")
else:
    print(f"Dataset size is exactly {TARGET_ROWS} rows after cleaning. No truncation needed.")


print("\n--- Final Cleaned Data Summary ---")
print(f"Final shape of the super-cleaned DataFrame: {df_cleaned.shape}")

print("\nFirst 5 rows of the super-cleaned data:")
print(df_cleaned.head().to_markdown(index=False, numalign="left", stralign="left"))

SUPER_CLEANED_FILE_NAME = 'super_cleaned_startup_data_for_prediction.csv'
df_cleaned.to_csv(SUPER_CLEANED_FILE_NAME, index=False)
print(f"\nSuper-cleaned data saved to: {SUPER_CLEANED_FILE_NAME}")


Starting shape for advanced cleaning: (52551, 10)
Shape after removing names starting with non-letters: (52093, 10) (458 rows removed)
Shape after removing numeric state codes: (39606, 10) (12487 rows removed)
Shape after removing single-letter names: (39606, 10) (0 rows removed)
Shape after validating/cleaning funding values: (34069, 10) (5537 rows removed)
Dataset truncated to the top 3000 rows.

--- Final Cleaned Data Summary ---
Final shape of the super-cleaned DataFrame: (3000, 10)

First 5 rows of the super-cleaned data:
| name                   | homepage_url               | category_list                                            | funding_total_usd   | status    | country_code   | state_code   | region      | city          | funding_rounds   |
|:-----------------------|:---------------------------|:---------------------------------------------------------|:--------------------|:----------|:---------------|:-------------|:------------|:--------------|:-----------------|
| Ondin

In [ ]:
df_cleaned.shape

(3000, 10)

In [ ]:
import requests
import pandas as pd
import time
import numpy as np
import json
import re
from bs4 import BeautifulSoup
import os

TARGET_URL = 'https://www.cbinsights.com/research-unicorn-companies'
SCRAPED_FILE_NAME = 'super_cleaned_startup_data_for_prediction.csv'


INR_CONVERSION_RATE = 88.78

SCRAPED_COLUMNS = [
    'name',
    'homepage_url',
    'category_list',
    'funding_total_usd',
    'status',
    'country_code',
    'state_code',
    'region',
    'city',
    'funding_rounds'
]

scraped_data = []

def clean_funding_value(funding_str):
    """
    Converts money strings (e.g., '$1.2B', '$500M') to numerical INR.
    The final value is multiplied by the INR_CONVERSION_RATE.
    """
    if not isinstance(funding_str, str):
        return None

    text = funding_str.strip().replace('$', '').replace(',', '').upper()

    if not text:
        return None

    usd_value = None

    if 'B' in text:
        try:
            usd_value = float(text.replace('B', '').strip()) * 1_000_000_000
        except ValueError:
            pass

    elif 'M' in text:
        try:
            usd_value = float(text.replace('M', '').strip()) * 1_000_000
        except ValueError:
            pass

    elif 'K' in text:
        try:
            usd_value = float(text.replace('K', '').strip()) * 1_000
        except ValueError:
            pass

    else:
        try:
            usd_value = float(text)
        except ValueError:
            pass

    if usd_value is not None:

        return usd_value * INR_CONVERSION_RATE

    return None


def scrape_data(url):
    """
    Fetches the HTML page and scrapes the main table elements directly.
    """
    print(f"Attempting to fetch HTML and parse the main company table from: {url}...")

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Referer': 'https://www.google.com/',
    }

    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')


        main_content_div = soup.find('div', id='company_list_table')
        if not main_content_div:
             main_content_div = soup

        table = main_content_div.find('table')

        if not table:
            print("ERROR: Could not find the main company table using generic selectors.")
            return []

        tbody = table.find('tbody')
        if not tbody:
            print("ERROR: Found table but could not locate the tbody for rows.")
            return []

        rows = tbody.find_all('tr')

        if not rows:
            print("ERROR: Found the table but no company rows were present.")
            return []

        print(f"SUCCESS: Found {len(rows)} company rows. This might be the full list or the initial static content.")

        page_results = []
        for i, row in enumerate(rows):

            cols = row.find_all(['td', 'th'])

            if len(cols) < 7:
                continue
            name_cell = cols[0].find('a')
            company_name = name_cell.text.strip() if name_cell else cols[0].text.strip()


            valuation_raw = cols[1].text.strip()

            category_list_raw = cols[3].text.strip()

            country_raw = cols[4].text.strip()

            city_raw = cols[5].text.strip()

            sanitized_name = re.sub(r'[^a-z0-9]', '', company_name.lower())
            homepage_url_derived = f"www.{sanitized_name}.com"
            unicorn_info = {
                'name': company_name,
                'homepage_url': homepage_url_derived,
                'category_list': category_list_raw,
                'funding_total_usd': clean_funding_value(valuation_raw),
                'status': 'operating',
                'country_code': country_raw,
                'state_code': city_raw,
                'region': city_raw,
                'city': city_raw,
                'funding_rounds': 5
            }
            page_results.append(unicorn_info)

        return page_results

    except requests.exceptions.HTTPError as e:
        print(f"ERROR: HTTP error {response.status_code} fetching HTML data {url}: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []


print(f"Starting web scraping process using HTML parsing for the CB Insights list.")
print(f"NOTE: Funding amounts will be converted to INR using a rate of 1 USD = {INR_CONVERSION_RATE} INR.")

scraped_data = scrape_data(TARGET_URL)

if scraped_data:
    df_new = pd.DataFrame(scraped_data, columns=SCRAPED_COLUMNS)
    df_new.replace({np.nan: None, '': None}, inplace=True)


    df_existing = pd.DataFrame()

    if os.path.exists(SCRAPED_FILE_NAME):
        try:

            df_existing = pd.read_csv(SCRAPED_FILE_NAME)
            print(f"Found {len(df_existing)} existing records in {SCRAPED_FILE_NAME}.")
        except pd.errors.EmptyDataError:
            print("Existing file found but it was empty.")
        except Exception as e:

            print(f"Error reading existing file: {e}. Starting with only new data.")

    df_existing = df_existing[SCRAPED_COLUMNS] if not df_existing.empty else df_existing
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    initial_total = len(df_combined)

    df_combined.drop_duplicates(subset=['name'], keep='last', inplace=True)
    final_total = len(df_combined)

    df_combined.to_csv(SCRAPED_FILE_NAME, index=False)

    print("\n--- Scraping Complete ---")

    print(f"Successfully scraped {len(df_new)} new/updated records.")
    print(f"Combined and saved {final_total} unique records (total) to {SCRAPED_FILE_NAME}.")
    print(f"Total duplicates removed/overwritten: {initial_total - final_total}.")

    print("\n--- Sample of FINAL Saved Data (Funding in INR) ---")
    print(df_combined.tail().to_markdown(index=False))

else:
    print("No data was scraped. The website likely loads the table content using JavaScript that our script cannot execute. The manual copy-paste method remains the only guaranteed alternative.")


Starting web scraping process using HTML parsing for the CB Insights list.
NOTE: Funding amounts will be converted to INR using a rate of 1 USD = 88.78 INR.
Attempting to fetch HTML and parse the main company table from: https://www.cbinsights.com/research-unicorn-companies...
SUCCESS: Found 1303 company rows. This might be the full list or the initial static content.
Found 3000 existing records in super_cleaned_startup_data_for_prediction.csv.

--- Scraping Complete ---
Successfully scraped 1303 new/updated records.
Combined and saved 4279 unique records (total) to super_cleaned_startup_data_for_prediction.csv.
Total duplicates removed/overwritten: 24.

--- Sample of FINAL Saved Data (Funding in INR) ---
| name                | homepage_url               | category_list   |   funding_total_usd | status    | country_code   | state_code            | region                | city                  |   funding_rounds |
|:--------------------|:---------------------------|:----------------|--

In [ ]:
df_combined.shape

(4279, 10)

In [ ]:
import pandas as pd
import re
import numpy as np

FILE_TO_LOAD = 'super_cleaned_startup_data_for_prediction.csv'

try:
    df_cleaned = pd.read_csv(FILE_TO_LOAD)
except FileNotFoundError:
    print(f"Error: The file '{FILE_TO_LOAD}' was not found. Please run 'cb_insights_scraper.py' first.")
    exit()
except Exception as e:
    print(f"Error loading {FILE_TO_LOAD}: {e}")
    exit()

print(f"Starting shape for advanced cleaning: {df_cleaned.shape}")

initial_shape = df_cleaned.shape[0]
name_mask = df_cleaned['name'].astype(str).str.match(r'^[a-zA-Z]')
df_cleaned = df_cleaned[name_mask]
print(f"Shape after removing names starting with non-letters: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")


initial_shape = df_cleaned.shape[0]
numeric_state_mask = df_cleaned['state_code'].astype(str).str.isnumeric()

df_cleaned = df_cleaned[~numeric_state_mask]
print(f"Shape after removing numeric state codes: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")


initial_shape = df_cleaned.shape[0]
single_letter_mask = df_cleaned['name'].astype(str).str.len() == 1
df_cleaned = df_cleaned[~single_letter_mask]
print(f"Shape after removing single-letter names: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")



initial_shape = df_cleaned.shape[0]
df_cleaned['funding_total_usd'] = pd.to_numeric(
    df_cleaned['funding_total_usd'],
    errors='coerce'
)
# Now drop the rows that became NaN due to the coercion
df_cleaned.dropna(subset=['funding_total_usd'], inplace=True)
print(f"Shape after validating/cleaning funding values: {df_cleaned.shape} ({initial_shape - df_cleaned.shape[0]} rows removed)")


# --- 5. Reduce dataset to the 2000 most 'valid' (first) rows ---
TARGET_ROWS = 3000
initial_shape_trunc = df_cleaned.shape[0]

if initial_shape_trunc > TARGET_ROWS:
    df_cleaned = df_cleaned.head(TARGET_ROWS)
    print(f"Dataset truncated to the top {TARGET_ROWS} rows.")
elif initial_shape_trunc < TARGET_ROWS:
    print(f"Warning: Only {initial_shape_trunc} rows remain after cleaning. No truncation performed.")
else:
    print(f"Dataset size is exactly {TARGET_ROWS} rows after cleaning. No truncation needed.")


# --- Final Output and Save ---

print("\n--- Final Cleaned Data Summary ---")
print(f"Final shape of the super-cleaned DataFrame: {df_cleaned.shape}")

# Display the first few rows of the super-cleaned data
print("\nFirst 5 rows of the super-cleaned data:")
print(df_cleaned.head().to_markdown(index=False, numalign="left", stralign="left"))

# Save the super-cleaned DataFrame to a new CSV file
SUPER_CLEANED_FILE_NAME = 'super_cleaned_startup_data_for_prediction.csv'
# NOTE: This line is redundant since the file has the same name as the input,
# but we keep it here to ensure the final, cleaned data is saved.
df_cleaned.to_csv(SUPER_CLEANED_FILE_NAME, index=False)
print(f"\nSuper-cleaned data saved to: {SUPER_CLEANED_FILE_NAME}")


Starting shape for advanced cleaning: (4279, 10)
Shape after removing names starting with non-letters: (4268, 10) (11 rows removed)
Shape after removing numeric state codes: (4268, 10) (0 rows removed)
Shape after removing single-letter names: (4268, 10) (0 rows removed)
Shape after validating/cleaning funding values: (4268, 10) (0 rows removed)
Dataset truncated to the top 3000 rows.

--- Final Cleaned Data Summary ---
Final shape of the super-cleaned DataFrame: (3000, 10)

First 5 rows of the super-cleaned data:
| name                          | homepage_url               | category_list                                            | funding_total_usd   | status    | country_code   | state_code   | region      | city          | funding_rounds   |
|:------------------------------|:---------------------------|:---------------------------------------------------------|:--------------------|:----------|:---------------|:-------------|:------------|:--------------|:-----------------|
| Ondi

In [ ]:
df_cleaned.shape

(3000, 10)

In [ ]:
df_combined.head()

,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds
0,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,7.628510e+05,operating,CAN,BC,Vancouver,Vancouver,2
2,One Inc.,http://whatis1.com,Mobile,1.150050e+06,operating,USA,CA,SF Bay Area,San Francisco,3
3,ZenChef,http://zenchef.com/en/,Local Businesses|Restaurants,1.066177e+07,operating,FRA,A8,Paris,Paris,5
4,Redox,http://www.redoxengine.com,Health Care|Health Care Information Technology...,4.000000e+06,operating,USA,WI,Madison,Madison,2
5,One Block Off the Grid (1BOG),http://1bog.org,Clean Technology|Residential Solar,5.000000e+06,closed,USA,CA,SF Bay Area,San Francisco,1


#**Data Preprocessing and EDA**

In [ ]:
import pandas as pd
import numpy as np

# Define the file name used by the scraper and cleaner scripts
FILE_NAME = 'super_cleaned_startup_data_for_prediction.csv'

# --- 1. Load the Super Cleaned Data ---
try:
    df = pd.read_csv(FILE_NAME)
    print(f"Successfully loaded data for preprocessing. Initial shape: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: The file '{FILE_NAME}' was not found.")
    print("Please ensure you have run 'cb_insights_scraper.py' to generate the data.")
    # Exit gracefully if the critical file is missing
    exit()
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    exit()


# --- 2. Data Preprocessing (Cleaning and Type Conversion) ---

print("\n--- 2. Data Preprocessing (Cleaning and Type Conversion) ---")

# The goal here is to make sure our columns are ready for math or grouping.

# 2.1 Type Conversion and Missing Data (NaN) Handling for Numeric Features
# We force these columns to be numbers. If the conversion fails (e.g., finding text),
# it replaces that value with NaN (Not a Number).
df['funding_total_usd'] = pd.to_numeric(df['funding_total_usd'], errors='coerce')
df['funding_rounds'] = pd.to_numeric(df['funding_rounds'], errors='coerce')

# 2.2 Missing Data Handling for Text (Categorical) Features
# For text columns, we replace any missing values (NaN) with 'Unknown'
# so they can be grouped correctly later.
df['country_code'].fillna('Unknown Country', inplace=True)
df['category_list'].fillna('Unknown Industry', inplace=True)
df['region'].fillna('Unknown Region', inplace=True)

# 2.3 Dropping Critical Missing Rows
# We must remove any rows that still have missing values in our key number columns
# (funding and rounds) because we can't do math or prediction without them.
initial_rows = df.shape[0]
df.dropna(subset=['funding_total_usd', 'funding_rounds'], inplace=True)
rows_removed = initial_rows - df.shape[0]

print(f"Rows removed due to missing Funding/Rounds: {rows_removed}")
print(f"Data shape after cleaning and preparation: {df.shape}")


# --- 3. Exploratory Data Analysis (EDA) ---

print("\n--- 3.1 Descriptive Statistics (Numeric Features in INR) ---")
# This gives us a quick summary (count, average, min, max, median) of our key numbers.
print(df[['funding_total_usd', 'funding_rounds']].describe().to_markdown(numalign="left", stralign="left"))

print("\n--- 3.2 Categorical Analysis: Top 5 Countries and Industries ---")
# This shows us which categories appear most often in the dataset.
print("Top 5 Countries:")
print(df['country_code'].value_counts().head(5).to_markdown(numalign="left", stralign="left"))
print("\nTop 5 Industries (Category List):")
print(df['category_list'].value_counts().head(5).to_markdown(numalign="left", stralign="left"))


# --- 3.3 CRITICAL STEP: Creating the Binary Target Variable (0 or 1) ---

print("\n--- 3.3 Creating Binary Target: is_highly_funded ---")
# This is our 'Success' score for prediction. We are turning a continuous number (funding)
# into a simple YES (1) or NO (0) question for the model to learn.

# 1. Calculate the middle (median) funding amount across all companies.
median_funding = df['funding_total_usd'].median()

# 2. Create the binary target column:
# 1 if the company's funding is ABOVE the median, 0 if it is at or BELOW the median.
df['is_highly_funded'] = (df['funding_total_usd'] > median_funding).astype(int)

print(f"Median Funding Total (in INR) used as threshold: ₹{median_funding:,.0f}")
print(f"Target distribution: 1 (Highly Funded) count: {df['is_highly_funded'].sum()} / Total rows: {df.shape[0]}")
print("New binary column 'is_highly_funded' added to the DataFrame.")

# --- 4. Prepare for Next Step (Feature Engineering) ---
# Save the new DataFrame with the target variable, ready for the machine learning model.
df.to_csv('prepared_for_model.csv', index=False)
print("\nPrepared data saved to: prepared_for_model.csv (Ready for Feature Engineering)")


Successfully loaded data for preprocessing. Initial shape: (3000, 10)

--- 2. Data Preprocessing (Cleaning and Type Conversion) ---
Rows removed due to missing Funding/Rounds: 0
Data shape after cleaning and preparation: (3000, 10)

--- 3.1 Descriptive Statistics (Numeric Features in INR) ---
|       | funding_total_usd   | funding_rounds   |
|:------|:--------------------|:-----------------|
| count | 3000                | 3000             |
| mean  | 2.01838e+07         | 2.14833          |
| std   | 6.6516e+07          | 1.71191          |
| min   | 100                 | 1                |
| 25%   | 669238              | 1                |
| 50%   | 3.459e+06           | 1                |
| 75%   | 1.63568e+07         | 3                |
| max   | 2.39482e+09         | 15               |

--- 3.2 Categorical Analysis: Top 5 Countries and Industries ---
Top 5 Countries:
| country_code   | count   |
|:---------------|:--------|
| USA            | 2568    |
| GBR            | 211    

/tmp/ipython-input-2445352745.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country_code'].fillna('Unknown Country', inplace=True)
/tmp/ipython-input-2445352745.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

#**Model Building**

In [ ]:
import pandas as pd
# Libraries for splitting data, training the model, and checking performance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np

# Define the file name from the previous preprocessing step
FILE_NAME = 'prepared_for_model.csv'

# --- 1. Load Data ---
try:
    df = pd.read_csv(FILE_NAME)
    print(f"Successfully loaded data for modeling. Initial shape: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: The file '{FILE_NAME}' was not found.")
    print("Please ensure you have run 'data_preprocessing_eda.py' first.")
    exit()
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    exit()


# --- 1.5. Apply Domain Constraint and Rename Target ---

# Rename the target column to 'success' as requested
df.rename(columns={'is_highly_funded': 'success'}, inplace=True)

print("\n--- 1.5. Applying Domain Constraints ---")
# Constraint: A closed company cannot be considered successful (a positive prediction).
# We explicitly set the 'success' score to 0 for any row where status is 'closed'.
closed_count = df[df['status'] == 'closed'].shape[0]
df.loc[df['status'] == 'closed', 'success'] = 0
print(f"Applied constraint: Set 'success' to 0 for {closed_count} rows where status is 'closed'.")
print(f"Target column renamed to 'success'.")


# --- 2. Feature Selection ---

# X (Features): The columns the model will use to guess the outcome.
# We choose columns available to a young startup (location, industry, rounds) AND status.
feature_cols = [
    'category_list', 'country_code', 'state_code', 'region', 'city', 'funding_rounds', 'status'
]
X = df[feature_cols]

# y (Target): The column the model is trying to predict (our 0 or 1 success score).
# This is now the 'success' column.
y = df['success']

print(f"\nFeatures selected: {feature_cols}")


# --- 3. Feature Engineering (One-Hot Encoding) ---

print("\n--- 3. Feature Engineering: One-Hot Encoding (Converting Text to Numbers) ---")
# Machine learning models only understand numbers. This step creates a new column
# (with 0s and 1s) for every unique text value in the original columns.
# Example: 'country_code' turns into 'country_code_USA', 'country_code_CHN', etc.
X_encoded = pd.get_dummies(X, columns=feature_cols, drop_first=True)

print(f"Original feature columns: {len(feature_cols)}")
print(f"New encoded feature columns (numbers only): {X_encoded.shape[1]}")
print("Data is now numerical and ready for the model.")

Successfully loaded data for modeling. Initial shape: (3000, 11)

--- 1.5. Applying Domain Constraints ---
Applied constraint: Set 'success' to 0 for 216 rows where status is 'closed'.
Target column renamed to 'success'.

Features selected: ['category_list', 'country_code', 'state_code', 'region', 'city', 'funding_rounds', 'status']

--- 3. Feature Engineering: One-Hot Encoding (Converting Text to Numbers) ---
Original feature columns: 7
New encoded feature columns (numbers only): 2557
Data is now numerical and ready for the model.


In [ ]:

# --- 4. Data Splitting (Train/Test Split) ---

# We split the data so the model can learn from one part (Train) and be tested
# on the other part (Test) to see if it generalizes to new data.
# Libraries for splitting data, training the model, and checking performance
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier # Added this import
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set size: {X_train.shape[0]} rows (80% of data)")
print(f"Testing set size: {X_test.shape[0]} rows (20% of data)")



# --- 5. Hyperparameter Tuning (Finding the Best Random Forest Settings) ---

print("\n--- 5. Hyperparameter Tuning using GridSearchCV ---")
print("This step searches for the optimal parameters to maximize the F1-Score...")
#

# Define the model to tune (starting with a balanced class weight)
rf_base = RandomForestClassifier(random_state=42, class_weight='balanced')

# Define the grid of parameters to test. We try different numbers of trees (n_estimators)
# and different maximum depths of those trees (max_depth).
param_grid = {
    'n_estimators': [50, 100, 200], # Number of trees in the forest
    'max_depth': [5, 10, None] # Maximum depth of each tree (None means unlimited)
}

# Use GridSearchCV to automatically test every combination in the grid.
# We set scoring='f1' to ensure the model optimizes for the F1-Score (our primary success metric).
grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    scoring='f1',
    cv=3, # Use 3-fold cross-validation
    verbose=0,
    n_jobs=-1 # Use all available cores for speed
)

# Train the grid search on the training data
grid_search.fit(X_train, y_train)

# The best model found during the search
final_model = grid_search.best_estimator_

print("Hyperparameter tuning complete.")
print(f"BEST parameters found: {grid_search.best_params_}")



print("\n--- 5. Model Training: Two Classifiers ---")

# Model A: Random Forest (Excellent for complex, mixed data)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)
print("Random Forest Model training complete.")

# Evaluate Random Forest model
rf_y_pred = rf_model.predict(X_test)
rf_f1 = f1_score(y_test, rf_y_pred)
print(f"Random Forest F1-Score: {rf_f1:.4f}")

# Model B: K-Nearest Neighbors (KNN - Simple, distance-based model)
# We use a neighbor count of 5 as a starting point.
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
print("K-Nearest Neighbors (KNN) Model training complete.")

# Evaluate KNN model
knn_y_pred = knn_model.predict(X_test)
knn_f1 = f1_score(y_test, knn_y_pred)
print(f"K-Nearest Neighbors (KNN) F1-Score: {knn_f1:.4f}")


# --- 6.3 Final Model Comparison and Selection ---
print("\n--- Final Model Selection ---")
if rf_f1 >= knn_f1:
    best_model_name = "Random Forest Classifier"
    best_model = rf_model
    y_pred = rf_y_pred
    y_proba = rf_model.predict_proba(X_test)[:, 1]
    best_f1 = rf_f1
else:
    best_model_name = "K-Nearest Neighbors (KNN)"
    best_model = knn_model
    y_pred = knn_y_pred
    y_proba = knn_model.predict_proba(X_test)[:, 1]
    best_f1 = knn_f1

print(f"Based on the F1-Score, the BEST model is the: {best_model_name} (F1: {best_f1:.4f})")
print("We prioritize F1-Score over simple Accuracy because it ensures we find true winners (Recall) without making too many false positive predictions (Precision).")

# Assign the best model to 'model' for consistency with subsequent cells
model = best_model

# --- 7. Feature Importance (Why the model made the decision) ---
# Feature importance is only available for the Random Forest model.
print("\n--- 7. Feature Importance (Top 10 from Random Forest if selected) ---")
if best_model_name == "Random Forest Classifier":
    feature_importances = pd.Series(model.feature_importances_, index=X_encoded.columns)
    top_features = feature_importances.nlargest(10)
    print("Which input factors are most important for predicting 'Highly Funded' (1) (from Random Forest):")
    print(top_features.to_markdown(numalign="left", stralign="left"))
else:
    print("Feature importance is not directly available for K-Nearest Neighbors.")


Training set size: 2400 rows (80% of data)
Testing set size: 600 rows (20% of data)

--- 5. Hyperparameter Tuning using GridSearchCV ---
This step searches for the optimal parameters to maximize the F1-Score...


#F1-score, confusion matrix (for classification)

In [ ]:

# 6.1 F1-Score and Classification Report
# F1-Score measures the balance between Precision (how accurate is the '1' prediction?) and
# Recall (how many true '1's did the model find?).
f1 = f1_score(y_test, y_pred)

print(f"Model F1-Score on Test Data: {f1:.4f} (Closer to 1.0 is better)")
print("\nClassification Report (Focus on F1-score for class 1):")
print(classification_report(y_test, y_pred, target_names=['Moderately Funded (0)', 'Highly Funded (1)']))


# 6.2 Confusion Matrix
# This shows exactly where the model was right and wrong on the test data.
cm = confusion_matrix(y_test, y_pred)
# Unpack the matrix components: True Negatives, False Positives, False Negatives, True Positives
tn, fp, fn, tp = cm.ravel()

print("\nConfusion Matrix (Test Data):")
print("------------------------------------------")
print("Interpreting the Matrix:")
print(f"| True Negatives (Correctly guessed 0s): {tn:<10} | False Positives (Wrongly guessed 1s): {fp:<10} |")
print(f"| False Negatives (Missed 1s): {fn:<10} | True Positives (Correctly guessed 1s): {tp:<10} |")
print("------------------------------------------")

# --- 7. Feature Importance (Why the model made the decision) ---
print("\n--- 7. Feature Importance (Top 10) ---")

# Feature importance tells us which input factors (Country, City, Round number, Industry)
# the model considered most important when making a prediction.
feature_importances = pd.Series(model.feature_importances_, index=X_encoded.columns)
top_features = feature_importances.nlargest(10)

print("Most important for predicting success (1):")
print(top_features.to_markdown(numalign="left", stralign="left"))